# 일등이조 (신경민, 현지영)
## 국방일보 나라별(중국, 미국, 러시아, 북한) 기사 크롤링 코드

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests
from urllib.request import urlopen
import time

In [2]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = 'https://kookbang.dema.mil.kr/newsWeb/allToday.do'
driver.get(url)

### 국방일보 뉴스 크롤링 코드(기사의 제목, 날짜, URL 수집)
- 대상 카테고리 : 국방('NKOOKBANG'), 국정일반('NGOVERNMENT'), 기획연재('NPLAN'), 오피니언('NOPINION')
- 수집 대상 데이터 : 기사제목(name_list), 게시날짜(year_list), 기사URL(url_list)

In [31]:
name_list = []
year_list = []  
url_list = []

categories = ['NKOOKBANG', 'NGOVERNMENT', 'NPLAN', 'NOPINION']

for category in categories:
    driver.execute_script(f"fn_searchType('{category}')")
    time.sleep(2)

    page = 1
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        article = soup.find_all('div', {'class': 'box'})
        for art in article:
            cart = art.find('h3')  # 카테고리 제목
            items = art.find_all('li')  # 각 기사 항목은 <li> 안에 있는 경우가 많음

            for item in items:
                title_tag = item.find('h3')
                date_tag = item.find('span')
                link_tag = item.find('a')

                # 예외 처리로 누락 방지
                if title_tag and date_tag and link_tag:
                    name_list.append(title_tag.text.strip() + ' <' + cart.text.strip() + '>')
                    year_list.append(date_tag.text.strip()[:13])
                    url_list.append('https://kookbang.dema.mil.kr' + link_tag.get('href'))

        # 페이지 넘기기
        page += 1
        try:
            next_page = driver.find_element(By.XPATH, f"//a[contains(@onclick, 'fn_search({page})')]")
            if next_page.is_displayed():
                driver.execute_script("arguments[0].scrollIntoView();", next_page)
                next_page.click()
                print(f"{category}: {page}페이지 이동")
                time.sleep(2)
            else:
                print(f"{category}: 페이지 끝 → 카테고리 종료")
                break
        except:
            print(f"{category}: 다음 페이지 없음 → 카테고리 종료")
            break

NKOOKBANG: 2페이지 이동
NKOOKBANG: 3페이지 이동
NKOOKBANG: 4페이지 이동
NKOOKBANG: 5페이지 이동
NKOOKBANG: 6페이지 이동
NKOOKBANG: 7페이지 이동
NKOOKBANG: 8페이지 이동
NKOOKBANG: 9페이지 이동
NKOOKBANG: 10페이지 이동
NKOOKBANG: 11페이지 이동
NKOOKBANG: 12페이지 이동
NKOOKBANG: 13페이지 이동
NKOOKBANG: 14페이지 이동
NKOOKBANG: 15페이지 이동
NKOOKBANG: 16페이지 이동
NKOOKBANG: 17페이지 이동
NKOOKBANG: 18페이지 이동
NKOOKBANG: 19페이지 이동
NKOOKBANG: 20페이지 이동
NKOOKBANG: 21페이지 이동
NKOOKBANG: 22페이지 이동
NKOOKBANG: 23페이지 이동
NKOOKBANG: 24페이지 이동
NKOOKBANG: 25페이지 이동
NKOOKBANG: 26페이지 이동
NKOOKBANG: 다음 페이지 없음 → 카테고리 종료
NGOVERNMENT: 2페이지 이동
NGOVERNMENT: 3페이지 이동
NGOVERNMENT: 4페이지 이동
NGOVERNMENT: 5페이지 이동
NGOVERNMENT: 6페이지 이동
NGOVERNMENT: 7페이지 이동
NGOVERNMENT: 8페이지 이동
NGOVERNMENT: 9페이지 이동
NGOVERNMENT: 10페이지 이동
NGOVERNMENT: 11페이지 이동
NGOVERNMENT: 12페이지 이동
NGOVERNMENT: 13페이지 이동
NGOVERNMENT: 14페이지 이동
NGOVERNMENT: 15페이지 이동
NGOVERNMENT: 16페이지 이동
NGOVERNMENT: 다음 페이지 없음 → 카테고리 종료
NPLAN: 2페이지 이동
NPLAN: 3페이지 이동
NPLAN: 4페이지 이동
NPLAN: 5페이지 이동
NPLAN: 6페이지 이동
NPLAN: 다음 페이지 없음 → 카테고리 종료
NOPINION: 2페이지 이동
NO

In [33]:
# 기사 이름, 기사 작성 날짜, 기사 url 주소 개수 확인
len(name_list), len(year_list), len(url_list)

(806, 806, 806)

### 웹 페이지에서 기사 본문 추출
- 각 URL에 있는 HTML 소스 코드를 읽어 BeautifulSoup으로 파싱
- 최종적으로 정제된 본문 내용 리스트에 저장

In [34]:
con_list = []
for i in url_list:
    try:
        html = urlopen(i).read()
        soup = BeautifulSoup(html, 'html.parser')
        contents = soup.find('div', {'id': 'article_body_view'})
        if contents:
            all_p = contents.find_all('p')
            text = ' '.join([
                p.text.strip() for p in all_p
                if not any(bad in p.text for bad in ['페이스북', '트위터', '밴드', '카카오톡', 'url복사'])
            ])
            for bad in ['페이스북', '트위터', '밴드', '카카오톡', 'url복사', '기자', '글=', '사진']:
                text = text.replace(bad, '')
            con_list.append(text.strip())
    except Exception as e:
        print(f'에러 발생: {e}')
        con_list.append('에러 발생')

### csv 파일 저장

In [ ]:
df = pd.DataFrame({
    'Date': year_list,
    'Title': name_list,
    'content':con_list
})
df.to_csv('16_북한기사.csv', index=False, encoding='utf-8-sig')